In [171]:
import pandas as pd
import numpy as np
import os
import getpass
import re
import nltk
import tqdm
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [170]:
#!pip install vaderSentiment
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import jupyterthemes as jt

In [3]:
df = pd.read_feather("tweets.ftr")

In [4]:
df

,timestamp,replies,likes,retweets,text
0,2019-05-27 11:49:14+00,0,0,0,È appena uscito un nuovo video! LES CRYPTOMONN...
1,2019-05-27 11:49:18+00,0,0,0,Cardano: Digitize Currencies; EOS https://t.co...
2,2019-05-27 11:49:06+00,0,2,1,Another Test tweet that wasn't caught in the s...
3,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,2019-05-27 11:49:23+00,0,0,0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...
...,...,...,...,...,...
20165008,2019-11-23 15:28:50+00,2,16,3,Happy #FibonacciDay \n\nA while back I created...
20165009,2019-11-23 15:45:55+00,0,0,0,Bitcoin Suisse Certificates :) https://t.co/nd...
20165010,2019-11-23 15:45:56+00,0,0,0,Register now for the early access of the Codug...
20165011,2019-11-23 15:45:57+00,0,0,0,@btc \n@btc \nDo you know that BTC Baskets isn...


In [5]:
df["timestamp"].describe()

count                   18810521
unique                  11127706
top       2018-08-03 12:01:15+00
freq                       65463
Name: timestamp, dtype: object

In [6]:
df[df['timestamp'] < "2018-08-03 12:01:15+00"]

,timestamp,replies,likes,retweets,text
21,2009-01-11 03:33:52+00,790,14470,5542,Running bitcoin
803,2017-09-08 15:45:33+00,3,5,3,How China Sinks Bitcoin - Probably what happen...
860,2018-04-17 14:44:58+00,1,7,1,Great talk and discussions about Bitcoin vs. E...
1599,2018-02-08 18:22:44+00,23,135,51,"When things are down, remember it's never as b..."
1784,2017-11-28 07:33:27+00,12,509,238,1/ While there are no a priori rules about the...
...,...,...,...,...,...
20154543,2013-04-17 16:03:15+00,0,2,2,Just who is the mysterious pseudonymous develo...
20155959,2017-03-19 10:00:01+00,0,0,9,Bitcoin 101: Four Ways Blockchain Can Disrupt ...
20158182,2018-05-21 14:57:13+00,0,1,1,Inbox me for BITCOIN investment https://t.co/s...
20158359,2018-05-15 15:11:53+00,0,1,1,Why don't you invest now for the future Bitcoi...


In [7]:
type(df["timestamp"][0])

str

In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [9]:
df

,timestamp,replies,likes,retweets,text
0,2019-05-27 11:49:14+00:00,0,0,0,È appena uscito un nuovo video! LES CRYPTOMONN...
1,2019-05-27 11:49:18+00:00,0,0,0,Cardano: Digitize Currencies; EOS https://t.co...
2,2019-05-27 11:49:06+00:00,0,2,1,Another Test tweet that wasn't caught in the s...
3,2019-05-27 11:49:22+00:00,0,0,0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,2019-05-27 11:49:23+00:00,0,0,0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...
...,...,...,...,...,...
20165008,2019-11-23 15:28:50+00:00,2,16,3,Happy #FibonacciDay \n\nA while back I created...
20165009,2019-11-23 15:45:55+00:00,0,0,0,Bitcoin Suisse Certificates :) https://t.co/nd...
20165010,2019-11-23 15:45:56+00:00,0,0,0,Register now for the early access of the Codug...
20165011,2019-11-23 15:45:57+00:00,0,0,0,@btc \n@btc \nDo you know that BTC Baskets isn...


In [10]:
df = df.sort_values(by='timestamp')

In [11]:
df

,timestamp,replies,likes,retweets,text
14404079,2007-04-19 07:14:38+00:00,0,0,2,is happily mugging at BTC where she will hook ...
21,2009-01-11 03:33:52+00:00,790,14470,5542,Running bitcoin
7563298,2009-01-21 17:29:40+00:00,55,1544,392,Looking at ways to add more anonymity to bitcoin
7609229,2009-01-27 20:14:10+00:00,44,1042,277,Thinking about how to reduce CO2 emissions fro...
7668664,2009-01-29 13:37:53+00:00,0,28,16,From: Satoshi Nakamoto - 2009-01-11 22:32 Bitc...
...,...,...,...,...,...
19956031,NaT,None,None,None,None
19956032,NaT,None,None,None,None
19956033,NaT,None,None,None,None
20062116,NaT,None,None,None,None


In [199]:
dff = df.dropna()

In [200]:
dff = dff.reset_index(drop = True)

In [201]:
dff

,timestamp,replies,likes,retweets,text
0,2007-04-19 07:14:38+00:00,0,0,2,is happily mugging at BTC where she will hook ...
1,2009-01-11 03:33:52+00:00,790,14470,5542,Running bitcoin
2,2009-01-21 17:29:40+00:00,55,1544,392,Looking at ways to add more anonymity to bitcoin
3,2009-01-27 20:14:10+00:00,44,1042,277,Thinking about how to reduce CO2 emissions fro...
4,2009-01-29 13:37:53+00:00,0,28,16,From: Satoshi Nakamoto - 2009-01-11 22:32 Bitc...
...,...,...,...,...,...
18809795,2019-11-23 15:45:53+00:00,0,0,0,The Worlds First Millennium Firm https://t.co/...
18809796,2019-11-23 15:45:55+00:00,0,0,0,Daily profit for HODLING BTC since 2013 Data t...
18809797,2019-11-23 15:45:55+00:00,0,0,0,Bitcoin Suisse Certificates :) https://t.co/nd...
18809798,2019-11-23 15:45:56+00:00,0,0,0,Register now for the early access of the Codug...


In [202]:
small_df = dff[:900]
small_df

,timestamp,replies,likes,retweets,text
0,2007-04-19 07:14:38+00:00,0,0,2,is happily mugging at BTC where she will hook ...
1,2009-01-11 03:33:52+00:00,790,14470,5542,Running bitcoin
2,2009-01-21 17:29:40+00:00,55,1544,392,Looking at ways to add more anonymity to bitcoin
3,2009-01-27 20:14:10+00:00,44,1042,277,Thinking about how to reduce CO2 emissions fro...
4,2009-01-29 13:37:53+00:00,0,28,16,From: Satoshi Nakamoto - 2009-01-11 22:32 Bitc...
...,...,...,...,...,...
895,2012-01-07 22:31:59+00:00,0,0,0,Sat Jan 7 23:31:00 2012 - Traded 0.01006 BTC ...
896,2012-01-07 22:32:00+00:00,0,0,0,Sat Jan 7 23:31:00 2012 - Traded 1.00964036 B...
897,2012-01-07 22:32:01+00:00,0,0,0,Sat Jan 7 23:31:00 2012 - Traded 32.78 BTC at...
898,2012-01-07 22:32:02+00:00,0,0,0,Sat Jan 7 23:31:00 2012 - Traded 2 BTC at 6.7...


In [189]:
small_df[["timestamp", "text"]].groupby(pd.Grouper(key='timestamp', freq='1Y')).count()

,text
timestamp,
2007-12-31 00:00:00+00:00,1
2008-12-31 00:00:00+00:00,0
2009-12-31 00:00:00+00:00,14
2010-12-31 00:00:00+00:00,25
2011-12-31 00:00:00+00:00,193
2012-12-31 00:00:00+00:00,667


In [207]:
small_df.loc[19]

timestamp                         2010-07-20 15:58:28+00:00
replies                                                   4
likes                                                    60
retweets                                                 13
text          Anyone using Bitcoin? http://www.bitcoin.org/
clean_text                             Anyone using Bitcoin
Name: 19, dtype: object

In [21]:
pd.options.mode.chained_assignment = None 

In [204]:
small_df['clean_text'] = small_df['text'].str.replace('http\S+|www.\S+', '', case=False) #removing any links

In [205]:
small_df['clean_text'] = small_df['clean_text'].str.replace('[^A-Za-z\s]+', '') #removing special chars

In [206]:
stop = stopwords.words('english')
small_df['clean_text'] = small_df['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) #removing stop words

In [116]:
#w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
#lemmatizer = nltk.stem.WordNetLemmatizer()

#def lemmatize_text(text): #lemmatize
#    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

#small_df['clean_text'] = small_df.clean_text.apply(lemmatize_text)

In [115]:
#stemmer = SnowballStemmer("english")
#small_df['clean_text'] = small_df['clean_text'].apply(lambda x: [stemmer.stem(y) for y in x]) #stemming

In [208]:
analyser = SentimentIntensityAnalyzer()
scores = []
for i in range(len(small_df['clean_text'])):
    score = analyser.polarity_scores(small_df['clean_text'][i])
    scores.append(score)

# for i in scores:
#     if i>=0.05:
#         sentiment.append('Positive')
#     elif i<=(-0.05):
#         sentiment.append('Negative')
#     else:
#         sentiment.append('Neutral')
# small_df['sentiment']=pd.Series(np.array(sentiment))

In [209]:
small_df[["neg", "nuet", "pos", "compound"]] = pd.DataFrame(scores)

In [210]:
small_df[small_df['compound'] > 0]

,timestamp,replies,likes,retweets,text,clean_text,neg,nuet,pos,compound
0,2007-04-19 07:14:38+00:00,0,0,2,is happily mugging at BTC where she will hook ...,happily mugging BTC hook future rich lawyer bf...,0.11,0.413,0.477,0.8402
7,2009-06-19 20:07:37+00:00,1,16,8,Very pleasant day cycling around London and co...,Very pleasant day cycling around London cowork...,0.00,0.754,0.246,0.5563
8,2009-06-19 21:19:07+00:00,0,15,6,"Hmm, @bitcoin looks interesting: any idea when...",Hmm bitcoin looks interesting idea something a...,0.00,0.722,0.278,0.4019
9,2009-06-19 23:40:39+00:00,0,12,5,@knaveofdiamonds We're getting ready to go int...,knaveofdiamonds Were getting ready go beta pre...,0.00,0.573,0.427,0.8020
10,2009-06-20 14:32:53+00:00,0,11,4,@afternoon not yet: am building an app during ...,afternoon yet building app may expand somethin...,0.00,0.796,0.204,0.3182
...,...,...,...,...,...,...,...,...,...,...
861,2012-01-06 14:55:52+00:00,0,0,0,For BTC 1.00: STEAM GAMES & MORE for BTC - GO...,For BTC STEAM GAMES MORE BTC GOOD REP TEAM FOR...,0.00,0.705,0.295,0.6816
862,2012-01-06 14:56:07+00:00,0,0,0,For BTC 1.00: Over 80 Top Level Domains - BUY ...,For BTC Over Top Level Domains BUY WITH BITCOI...,0.00,0.859,0.141,0.2023
865,2012-01-06 19:56:52+00:00,0,0,0,@Gorgeouss_Btc \nYou better . I don't get home...,GorgeoussBtc You better I dont get home till like,0.00,0.565,0.435,0.6597
870,2012-01-07 08:00:02+00:00,0,0,0,"Current BTC value: USD: 6.29/7.31, EUR: 4.94/5...",Current BTC value USD EUR GBP CHF SLL bitcoin,0.00,0.769,0.231,0.3400


### ***FinalScore = CompoundScore ∗ UserFollowerCount ∗ (Likes +1)∗(RetweetCount +1) - UserFollowerCount Not Available.***

In [211]:
small_df["replies"] = pd.to_numeric(small_df.replies, errors='coerce')
small_df["likes"] = pd.to_numeric(small_df.likes, errors='coerce')
small_df["retweets"] = pd.to_numeric(small_df.retweets, errors='coerce')

In [212]:
small_df["FinalScore"] = small_df["compound"] * ((small_df["likes"]) + 1) * ((small_df["retweets"]) + 1) 

In [218]:
small_df[small_df['compound'] > 0].sample(n=100)

,timestamp,replies,likes,retweets,text,clean_text,neg,nuet,pos,compound,FinalScore
148,2011-06-18 09:15:44+00:00,0,0,1,What happens to the intrinsic value of Bitcoin...,What happens intrinsic value Bitcoin get k Bit...,0.000,0.769,0.231,0.3400,0.6800
45,2011-04-16 20:18:13+00:00,0,1,1,researching this @bitcoin matter,researching bitcoin matter,0.000,0.645,0.355,0.0258,0.1032
50,2011-04-29 17:42:03+00:00,99,542,256,Just sold 100 #bitcoins for $220 cash. I trip...,Just sold bitcoins cash I tripled investment B...,0.000,0.699,0.301,0.6808,95006.3208
135,2011-06-13 05:34:29+00:00,3,1,8,"Gavin Andresen, of Bitcoin, is coming straight...",Gavin Andresen Bitcoin coming straight The Bit...,0.000,0.881,0.119,0.2263,4.0734
18,2010-07-19 15:01:48+00:00,1,35,15,"So far, only created one bitcoin block. Long, ...",So far created one bitcoin block Long slow gru...,0.138,0.569,0.293,0.3134,180.5184
...,...,...,...,...,...,...,...,...,...,...,...
821,2012-01-04 23:58:57+00:00,0,0,0,"#bitcoin just hit $6.00, nice rally here...",bitcoin hit nice rally,0.000,0.517,0.483,0.4215,0.4215
146,2011-06-17 19:59:48+00:00,0,4,9,dont let #bitcoin forum flamers manipulate you...,dont let bitcoin forum flamers manipulate Time...,0.000,0.791,0.209,0.4404,22.0200
33,2010-12-07 03:42:55+00:00,0,1,1,@aaronsw Other people are obsessed with hyperd...,aaronsw Other people obsessed hyperdeflation W...,0.126,0.741,0.133,0.0258,0.1032
111,2011-06-08 22:47:04+00:00,2,8,6,"sold out of my bitcoin ""investment"" at 2x\n\nwin.",sold bitcoin investment x win,0.000,0.513,0.487,0.5859,36.9117
